In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

bc=datasets.load_breast_cancer()
x,y=bc.data,bc.target
ns,nf=x.shape
print(ns,nf)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2)

#Always use standard scaler when using logistic regression because it scales the data around a mean of 0s
sc=StandardScaler()
#use only transform for test data because fit calculates mean and variance 
#and you want that to be a surprise to your model, so you use the training mean and variance
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)

xtrain=torch.from_numpy(xtrain.astype(np.float32))
xtest=torch.from_numpy(xtest.astype(np.float32))
ytrain=torch.from_numpy(ytrain.astype(np.float32))
ytest=torch.from_numpy(ytest.astype(np.float32))

569 30


In [2]:
ytrain=ytrain.view(ytrain.shape[0],1)
ytest=ytest.view(ytest.shape[0],1)

In [3]:
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear=nn.Linear(n_input_features,1)
    
    def forward(self,x):
        ypred=torch.sigmoid(self.linear(x))
        return ypred

In [4]:
model=LogisticRegression(nf)

criterion=nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

n_epochs=100

In [5]:
for epoch in range(n_epochs):
    ypred=model(xtrain)
    loss=criterion(ypred,ytrain)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if(epoch+1%10)==1:
        print(loss.item())

0.8113318681716919


In [13]:
print(ytest[30])

tensor([1.])


In [14]:
#use no grad when predicting because otherwise it will calculate the gradient
with torch.no_grad():
    print(model(xtest[30]).round())

tensor([1.])
